In [1]:
import gensim
from gensim import corpora
import math

In [2]:
class BM25 :
    def __init__(self, fn_docs, delimiter='|') :
        self.dictionary = corpora.Dictionary()
        self.DF = {}
        self.delimiter = delimiter
        self.DocTF = []
        self.DocIDF = {}
        self.N = 0
        self.DocAvgLen = 0
        self.fn_docs = fn_docs
        self.DocLen = []
        self.buildDictionary()
        self.TFIDF_Generator()
    
    def buildDictionary(self) :
        raw_data = []
        for line in self.fn_docs :
            raw_data.append(line.strip().split(self.delimiter))
        self.dictionary.add_documents(raw_data)
    
    def TFIDF_Generator(self, base=10) :
        docTotalLen = 0
        for line in self.fn_docs :
            doc = line.strip().split(self.delimiter)
            docTotalLen += len(doc)
            self.DocLen.append(len(doc))
            #print self.dictionary.doc2bow(doc)
            bow = dict([(term, freq*1.0/len(doc)) for term, freq in self.dictionary.doc2bow(doc)])
            for term, tf in bow.items() :
                if term not in self.DF :
                    self.DF[term] = 0
                self.DF[term] += 1
            self.DocTF.append(bow)
            self.N = self.N + 1
        for term in self.DF:
            #self.DocIDF[term] = math.log((self.N - self.DF[term] +0.5) / (self.DF[term] + 0.5), base)
            #print("no. docs:", self.N)
            #print("df:", self.DF[term])
            self.DocIDF[term] = math.log((self.N +1) / self.DF[term], base)
            #print("IDF:", self.DocIDF[term])
        self.DocAvgLen = docTotalLen / self.N
        #print("avg:", self.DocAvgLen)

    def BM25Score(self, Query=[], k1=1.5, b=0.75) :
        query_bow = self.dictionary.doc2bow(Query)
        scores = []
        for idx, doc in enumerate(self.DocTF) :
            commonTerms = set(dict(query_bow).keys()) & set(doc.keys())
            tmp_score = []
            doc_terms_len = self.DocLen[idx]
            for term in commonTerms :
                upper = (doc[term] * (k1+1))
                below = ((doc[term]) + k1*(1 - b + b*doc_terms_len/self.DocAvgLen))
                tmp_score.append(self.DocIDF[term] * upper / below)
            scores.append(sum(tmp_score))
        return scores
    
    def TFIDF(self) :
        tfidf = []
        for doc in self.DocTF :
            doc_tfidf  = [(term, tf*self.DocIDF[term]) for term, tf in doc.items()]
            doc_tfidf.sort()
            tfidf.append(doc_tfidf)
        return tfidf

    def Items(self) :
        # Return a list [(term_idx, term_desc),]
        items = self.dictionary.items()
        #items.sort()
        return items

In [3]:
fn_docs = ["Human machine interface for lab abc computer applications",
           "A survey of user opinion of computer system response time",
           "The EPS user interface management system",
           "System and human system engineering testing of EPS",
           "Relation of user perceived response time to error measurement",
           "The generation of random binary unordered trees",
           "The intersection graph of paths in trees",
           "Graph minors IV Widths of trees and well quasi ordering",
           "Graph minors A survey"]

In [4]:
bm25 = BM25(fn_docs, delimiter=' ')

In [5]:
Query = 'The intersection graph of paths in trees survey Graph'
Query = Query.split()
print(Query)

['The', 'intersection', 'graph', 'of', 'paths', 'in', 'trees', 'survey', 'Graph']


In [11]:
#scores = bm25.BM25Score(Query)
scores = bm25.BM25Score(fn_docs[0].split())
print(scores)

[1.381108930190397, 0.08996256351275003, 0.2047937592322093, 0, 0, 0, 0, 0, 0]


In [7]:
tfidf = bm25.TFIDF()
print(tfidf)

[[(0, 0.125), (1, 0.125), (2, 0.125), (3, 0.08737125054200234), (4, 0.125), (5, 0.08737125054200234), (6, 0.125), (7, 0.125)], [(3, 0.06989700043360188), (8, 0.06989700043360188), (9, 0.04436974992327128), (10, 0.1), (11, 0.06989700043360188), (12, 0.06989700043360188), (13, 0.05228787452803377), (14, 0.06989700043360188), (15, 0.05228787452803377)], [(5, 0.11649500072266979), (13, 0.08714645754672293), (15, 0.08714645754672293), (16, 0.11649500072266979), (17, 0.08714645754672293), (18, 0.16666666666666666)], [(9, 0.027731093702044546), (13, 0.0653598431600422), (16, 0.08737125054200234), (19, 0.125), (20, 0.08737125054200234), (21, 0.125), (22, 0.125), (23, 0.125)], [(9, 0.02464986106848404), (11, 0.07766333381511319), (14, 0.07766333381511319), (15, 0.05809763836448195), (24, 0.1111111111111111), (25, 0.1111111111111111), (26, 0.1111111111111111), (27, 0.1111111111111111), (28, 0.1111111111111111)], [(9, 0.03169267851662234), (17, 0.0746969636114768), (29, 0.14285714285714285), (30,

In [8]:
bm25.Items()

ItemsView(<gensim.corpora.dictionary.Dictionary object at 0x7f81467ca358>)

In [9]:
for i, tfidfscore in enumerate(tfidf):
    print(fn_docs[i], tfidfscore)

Human machine interface for lab abc computer applications [(0, 0.125), (1, 0.125), (2, 0.125), (3, 0.08737125054200234), (4, 0.125), (5, 0.08737125054200234), (6, 0.125), (7, 0.125)]
A survey of user opinion of computer system response time [(3, 0.06989700043360188), (8, 0.06989700043360188), (9, 0.04436974992327128), (10, 0.1), (11, 0.06989700043360188), (12, 0.06989700043360188), (13, 0.05228787452803377), (14, 0.06989700043360188), (15, 0.05228787452803377)]
The EPS user interface management system [(5, 0.11649500072266979), (13, 0.08714645754672293), (15, 0.08714645754672293), (16, 0.11649500072266979), (17, 0.08714645754672293), (18, 0.16666666666666666)]
System and human system engineering testing of EPS [(9, 0.027731093702044546), (13, 0.0653598431600422), (16, 0.08737125054200234), (19, 0.125), (20, 0.08737125054200234), (21, 0.125), (22, 0.125), (23, 0.125)]
Relation of user perceived response time to error measurement [(9, 0.02464986106848404), (11, 0.07766333381511319), (14,